In [1]:
import pyforest
import sys
sys.path.append('/Users/jayceepang/msse/ATOM_CODE/UTILS/')
from RF_functions import *
from GenUtils import *
from dataset import *
import os
import pickle

from imblearn.over_sampling import SMOTEN, ADASYN, SMOTE
from sklearn.metrics import accuracy_score, balanced_accuracy_score,precision_score, f1_score, roc_auc_score, roc_curve, precision_recall_curve, auc, recall_score, confusion_matrix,matthews_corrcoef

from rdkit import Chem
from rdkit.Chem import AllChem

In [5]:
neks = ['NEK2_binding','NEK2_inhibition','NEK3_binding','NEK5_binding','NEK9_binding','NEK9_inhibition']
samplings =['none_scaled','UNDER','SMOTE'] 

RF_types = ['RF','RF_BCW','BRFC','BRFC_BCW']
og_datapath = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/paper/datasets/80train_20test/featurized/'
results_dir='/Users/jayceepang/msse/ATOM_CODE/examples/holdout/cross_nek_results_20test/'
datapath = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/paper/datasets/80train_20test/featurized/'
rf_path = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/paper/results/RF_results/'
gp_path ='/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/paper/results/GP_results/'
# final_cols=['model','NEK','strategy','feat_type','RF_type', 'cm','recall', 'specificity', 'accuracy', 'precision', 
#                 'f1', 'ROC_AUC', 'MCC', 'balanced_accuracy']
feat_cols_dropped =['NEK','compound_id','base_rdkit_smiles', 'subset', 'active'] 
final_cols=['model','prod_model','data NEK','NEK', 'model NEK', 'strategy','feat_type','RF_type',
            'cm','recall', 'specificity', 'accuracy', 'precision', 
                'f1', 'ROC_AUC', 'MCC', 'balanced_accuracy', '1-specificity', 'model strategy',
           ]

In [9]:
test_results=[]

for nek in neks: 
    trainX=pd.read_csv(f'{og_datapath}{nek}_MFP_{samp}_trainX.csv')
    train_y =pd.read_csv(f'{og_datapath}{nek}_MFP_{samp}_train_y.csv').to_numpy().reshape(-1)
    
    for xnek in neks: 
        testX=pd.read_csv(f'{datapath}{xnek}_MFP_none_scaled.csv').drop(columns=feat_cols_dropped)
        test_y=pd.read_csv(f'{datapath}{xnek}_MFP_none_scaled.csv')['active'].to_numpy().reshape(-1)
        for samp in samplings: 
            for rf in RF_types: 
                model_name = f'{nek}_MFP_{samp}_{rf}'
                
                print(model_name, xnek) 
           
                model = rf_models(trainX, train_y, testX, test_y, rf, {}, True)
         
                test_df = gather_rf_results(model, testX, test_y)
                print()
    
                test_df['prod_model'] = f'{model_name}'
                test_df['model']=f'{nek}_{model_name}'
                test_df['data NEK'] =xnek
                test_df['model NEK'] = nek
                test_df['NEK']=nek
                test_df['feat_type'] = 'MFP'
                test_df['strategy'] = samp
                test_df['RF_type'] = rf
                test_df['model strategy'] = f'{samp}_{rf}'
                test_df['1-specificity']=1-test_df['specificity'] 
                
                test_df.to_csv(f'{results_dir}{model_name}_holdout_test.csv',index=False) 
                test_results.append(test_df.iloc[[0]][final_cols].values.flatten())

all_test =  pd.DataFrame(test_results,columns=final_cols)
all_test['modeling_type'] = 'RF' 
all_test['set'] = 'og_cross_nek' 
# all_test.to_csv(f'', index=False)                 
         

NEK2_binding_MFP_none_scaled_RF NEK2_binding
TRAIN: acc: 1.000, precision: 1.000, recall: 1.000, spec: 1.000
TEST: acc: 0.979, precision: 0.712, recall: 0.825, spec: 0.986
func:'rf_models'  took: 5.2569 sec

NEK2_binding_MFP_none_scaled_RF_BCW NEK2_binding
TRAIN: acc: 1.000, precision: 1.000, recall: 1.000, spec: 1.000
TEST: acc: 0.979, precision: 0.712, recall: 0.825, spec: 0.986
func:'rf_models'  took: 5.2279 sec

NEK2_binding_MFP_none_scaled_BRFC NEK2_binding


/Users/jayceepang/miniconda3/envs/ATOM_CODE/lib/python3.9/site-packages/imblearn/ensemble/_forest.py:577: FutureWarning: The default of `sampling_strategy` will change from `'auto'` to `'all'` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `'all'` to silence this warning and adopt the future behaviour.
  warn(
/Users/jayceepang/miniconda3/envs/ATOM_CODE/lib/python3.9/site-packages/imblearn/ensemble/_forest.py:589: FutureWarning: The default of `replacement` will change from `False` to `True` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `True` to silence this warning and adopt the future behaviour.
  warn(


TRAIN: acc: 1.000, precision: 1.000, recall: 1.000, spec: 1.000
TEST: acc: 0.979, precision: 0.701, recall: 0.825, spec: 0.985
func:'rf_models'  took: 9.3572 sec

NEK2_binding_MFP_none_scaled_BRFC_BCW NEK2_binding


/Users/jayceepang/miniconda3/envs/ATOM_CODE/lib/python3.9/site-packages/imblearn/ensemble/_forest.py:577: FutureWarning: The default of `sampling_strategy` will change from `'auto'` to `'all'` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `'all'` to silence this warning and adopt the future behaviour.
  warn(
/Users/jayceepang/miniconda3/envs/ATOM_CODE/lib/python3.9/site-packages/imblearn/ensemble/_forest.py:589: FutureWarning: The default of `replacement` will change from `False` to `True` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `True` to silence this warning and adopt the future behaviour.
  warn(


TRAIN: acc: 1.000, precision: 1.000, recall: 1.000, spec: 1.000
TEST: acc: 0.977, precision: 0.681, recall: 0.825, spec: 0.984
func:'rf_models'  took: 9.2595 sec

NEK2_binding_MFP_UNDER_RF NEK2_binding
TRAIN: acc: 1.000, precision: 1.000, recall: 1.000, spec: 1.000
TEST: acc: 0.977, precision: 0.681, recall: 0.825, spec: 0.984
func:'rf_models'  took: 5.4091 sec

NEK2_binding_MFP_UNDER_RF_BCW NEK2_binding
TRAIN: acc: 1.000, precision: 1.000, recall: 1.000, spec: 1.000
TEST: acc: 0.979, precision: 0.701, recall: 0.825, spec: 0.985
func:'rf_models'  took: 5.2550 sec

NEK2_binding_MFP_UNDER_BRFC NEK2_binding


...

TRAIN: acc: 0.998, precision: 1.000, recall: 0.996, spec: 1.000
TEST: acc: 0.987, precision: 0.951, recall: 0.929, spec: 0.994
func:'rf_models'  took: 0.9410 sec

NEK9_inhibition_MFP_SMOTE_RF NEK9_inhibition
TRAIN: acc: 0.998, precision: 1.000, recall: 0.996, spec: 1.000
TEST: acc: 0.987, precision: 0.951, recall: 0.929, spec: 0.994
func:'rf_models'  took: 0.6046 sec

NEK9_inhibition_MFP_SMOTE_RF_BCW NEK9_inhibition
TRAIN: acc: 0.998, precision: 1.000, recall: 0.996, spec: 1.000
TEST: acc: 0.987, precision: 0.951, recall: 0.929, spec: 0.994
func:'rf_models'  took: 0.6089 sec

NEK9_inhibition_MFP_SMOTE_BRFC NEK9_inhibition


/Users/jayceepang/miniconda3/envs/ATOM_CODE/lib/python3.9/site-packages/imblearn/ensemble/_forest.py:577: FutureWarning: The default of `sampling_strategy` will change from `'auto'` to `'all'` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `'all'` to silence this warning and adopt the future behaviour.
  warn(
/Users/jayceepang/miniconda3/envs/ATOM_CODE/lib/python3.9/site-packages/imblearn/ensemble/_forest.py:589: FutureWarning: The default of `replacement` will change from `False` to `True` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `True` to silence this warning and adopt the future behaviour.
  warn(


TRAIN: acc: 0.998, precision: 1.000, recall: 0.996, spec: 1.000
TEST: acc: 0.987, precision: 0.951, recall: 0.929, spec: 0.994
func:'rf_models'  took: 0.9284 sec

NEK9_inhibition_MFP_SMOTE_BRFC_BCW NEK9_inhibition


/Users/jayceepang/miniconda3/envs/ATOM_CODE/lib/python3.9/site-packages/imblearn/ensemble/_forest.py:577: FutureWarning: The default of `sampling_strategy` will change from `'auto'` to `'all'` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `'all'` to silence this warning and adopt the future behaviour.
  warn(
/Users/jayceepang/miniconda3/envs/ATOM_CODE/lib/python3.9/site-packages/imblearn/ensemble/_forest.py:589: FutureWarning: The default of `replacement` will change from `False` to `True` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `True` to silence this warning and adopt the future behaviour.
  warn(


TRAIN: acc: 0.998, precision: 1.000, recall: 0.996, spec: 1.000
TEST: acc: 0.987, precision: 0.951, recall: 0.929, spec: 0.994
func:'rf_models'  took: 0.9365 sec



In [10]:
all_test.to_csv(f'{results_dir}all_NEK_RF_cross_results.csv', index=False)  

In [11]:
import pyforest
import sys
sys.path.append('/Users/jayceepang/msse/ATOM_CODE/UTILS/')
from RF_functions import *
from GP_functions import *
from GenUtils import *
from dataset import *
import os
import pickle
from imblearn.over_sampling import SMOTEN, ADASYN, SMOTE
from sklearn.metrics import accuracy_score, balanced_accuracy_score,precision_score, f1_score, roc_auc_score, roc_curve, precision_recall_curve, auc, recall_score, confusion_matrix,matthews_corrcoef

from rdkit import Chem
from rdkit.Chem import AllChem
import math
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import os

import shutil
import sklearn
from sklearn.model_selection import KFold
import gpytorch
from gpytorch.models import ExactGP
from gpytorch.likelihoods import DirichletClassificationLikelihood
from gpytorch.means import ConstantMean
from gpytorch.kernels import ScaleKernel, RBFKernel, MaternKernel

from sklearn.metrics import confusion_matrix
import itertools
from sklearn.metrics import precision_score, recall_score, roc_auc_score, matthews_corrcoef, balanced_accuracy_score, confusion_matrix, f1_score, roc_curve,precision_recall_curve, auc
# from scipy.stats
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, f1_score, roc_auc_score, roc_curve, precision_recall_curve, auc, recall_score, confusion_matrix


# from RF_GSCV import * # RF_GSCV contains the calculate metrics function to get the TP, TN, FP, FN scores 
# from RF_atomver import prediction_type 


In [13]:
gp_path ='/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/paper/results/GP_results/'
feat_cols_dropped =['NEK','compound_id','base_rdkit_smiles', 'subset', 'active'] 

In [14]:
def load_and_predict(model_path, likelihood_path, testX, testy, kernel='matern'):
    """
    Get pickled model+likelihood and run predictions new datasets

    @args:
        model_path (str): path topickled model file
        likelihood_path (str): Path to the pickled likelihood file.
        testX (torch.Tensor), testy (torch.Tensor)
        kernel (str): Kernel type RBF or matern
    @returns
        test_perf_df (pd.DataFrame): Prediction results and metrics on test data.
    """
    with open(model_path, 'rb') as f:
        model = pickle.load(f)
    with open(likelihood_path, 'rb') as f:
        likelihood = pickle.load(f)

    trainer = Trainer(model, likelihood, iterations=300)
    test_dist, test_observed_pred, test_pred_means, test_pred = trainer.predict(testX)
    test_results = trainer.gp_results(testX, testy)

    test_pred_samples = test_dist.sample(torch.Size((100,))).exp()

    test_probabilities = (test_pred_samples / test_pred_samples.sum(-2, keepdim=True)).mean(0)
    test_prob_stds = (test_pred_samples / test_pred_samples.sum(-2, keepdim=True)).std(0)

    test_perf_df = pd.DataFrame()
    test_perf_df['mean_pred_class0'] = test_observed_pred.mean.numpy()[0,]
    test_perf_df['mean_pred_class1'] = test_observed_pred.mean.numpy()[1,]
    test_perf_df['y'] = testy
    test_perf_df['y_pred'] = test_pred_means.max(0)[1]
    test_perf_df['var_pred_class0'] = test_observed_pred.variance.numpy()[0,]
    test_perf_df['var_pred_class1'] = test_observed_pred.variance.numpy()[1,]
    test_perf_df['pred_prob_class0'] = test_probabilities.numpy()[0,]
    test_perf_df['pred_prob_class1'] = test_probabilities.numpy()[1,]
    test_perf_df['pred_prob_std_class0'] = test_prob_stds.numpy()[0,]
    test_perf_df['pred_prob_std_class1'] = test_prob_stds.numpy()[1,]
    test_perf_df['subset'] = 'nanobret_cellular_holdout'

    for k, val in test_results.items():
        test_perf_df[k] = val

    return test_perf_df


In [17]:
gp_test_results=[]
final_cols=['model','prod_model','NEK','data NEK', 'model NEK','strategy','feat_type', 'cm',
            'recall', 'specificity', 'accuracy', 'precision', 
                'f1', 'ROC_AUC', 'MCC', 'balanced_accuracy', '1-specificity', 'model strategy']
for nek in neks:
    trainX=pd.read_csv(f'{og_datapath}{nek}_MFP_{samp}_trainX.csv')
    train_y =pd.read_csv(f'{og_datapath}{nek}_MFP_{samp}_train_y.csv').to_numpy().reshape(-1)
    
    for xnek in neks: 
        # testX=pd.read_csv(f'{datapath}{xnek}_MFP_none_unscaled.csv').drop(columns=feat_cols_dropped)
        # test_y=pd.read_csv(f'{datapath}{xnek}_MFP_none_unscaled.csv')['active'].to_numpy().reshape(-1)
        testX, testy = make_torch_tens_float_simple(file_path_X=None,file_path_y=None, df_path=f'{datapath}{xnek}_MFP_none_unscaled.csv')
        for samp in samplings: 
            model_name = f'{nek}_MFP_none_scaled'
            print(model_name, xnek) 
            # _,_, testX,testy=make_torch_tens_float(filepath=None, filename=None, rootname=None, df=None)
            
            gp_model = f'{gp_path}{nek}_MFP_{samp}_matern.pkl'
     
            gp_likelihood = f'{gp_path}{nek}_MFP_{samp}_matern_likelihood.pkl'
            test_df = load_and_predict(gp_model,gp_likelihood, testX,testy, kernel='matern')
            test_df['model'] = f'{nek}_{xnek}_MFP_{samp}_GP_matern' # UNDER_GP_matern
            test_df['prod_model'] = f'{nek}_{samp}_GP_matern'
            test_df['data NEK'] =xnek
            test_df['model NEK'] =nek
            test_df['NEK']=nek
            test_df['feat_type'] = 'MFP'
            test_df['strategy'] = samp
            # test_df['kernel_type'] = 'GP_matern' 
            test_df['model strategy'] = f'{samp}_GP_matern'
            test_df['1-specificity']=1-test_df['specificity'] 
            test_df.to_csv(f'{results_dir}{model_name}_{samp}_GP_matern_cross_nek.csv', index=False)
            gp_test_results.append(test_df.iloc[[0]][final_cols].values.flatten())
            
    all_test =  pd.DataFrame(gp_test_results,columns=final_cols)
    all_test['modeling_type'] = 'GP' 
    all_test['set'] = 'og_cross_nek' 

    all_test.to_csv(f'{results_dir}GP_og_cross_nek_results.csv', index=False)                 
         

NEK2_binding_MFP_none_scaled NEK2_binding


/Users/jayceepang/miniconda3/envs/ATOM_CODE/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


accuracy: 0.9595, precision: 0.0000, recall: 0.0000, specificity: 1.0000, cm: [[1351    0]
 [  57    0]]
NEK2_binding_MFP_none_scaled NEK2_binding
accuracy: 0.8537, precision: 0.2055, recall: 0.9123, specificity: 0.8512, cm: [[1150  201]
 [   5   52]]
NEK2_binding_MFP_none_scaled NEK2_binding
accuracy: 0.9865, precision: 0.9524, recall: 0.7018, specificity: 0.9985, cm: [[1349    2]
 [  17   40]]
NEK2_binding_MFP_none_scaled NEK2_inhibition


/Users/jayceepang/miniconda3/envs/ATOM_CODE/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


accuracy: 0.9311, precision: 0.0000, recall: 0.0000, specificity: 1.0000, cm: [[1892    0]
 [ 140    0]]
NEK2_binding_MFP_none_scaled NEK2_inhibition
accuracy: 0.9001, precision: 0.3433, recall: 0.4929, specificity: 0.9302, cm: [[1760  132]
 [  71   69]]
NEK2_binding_MFP_none_scaled NEK2_inhibition
accuracy: 0.9291, precision: 0.3333, recall: 0.0286, specificity: 0.9958, cm: [[1884    8]
 [ 136    4]]
NEK2_binding_MFP_none_scaled NEK3_binding


...

/Users/jayceepang/miniconda3/envs/ATOM_CODE/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


accuracy: 0.9216, precision: 0.0000, recall: 0.0000, specificity: 1.0000, cm: [[1140    0]
 [  97    0]]
NEK9_inhibition_MFP_none_scaled NEK5_binding
accuracy: 0.9022, precision: 0.2778, recall: 0.1546, specificity: 0.9658, cm: [[1101   39]
 [  82   15]]
NEK9_inhibition_MFP_none_scaled NEK5_binding
accuracy: 0.9224, precision: 0.5455, recall: 0.0619, specificity: 0.9956, cm: [[1135    5]
 [  91    6]]
NEK9_inhibition_MFP_none_scaled NEK9_binding
accuracy: 0.9567, precision: 0.0000, recall: 0.0000, specificity: 1.0000, cm: [[1348    0]
 [  61    0]]
NEK9_inhibition_MFP_none_scaled NEK9_binding


/Users/jayceepang/miniconda3/envs/ATOM_CODE/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


accuracy: 0.9375, precision: 0.2632, recall: 0.2459, specificity: 0.9688, cm: [[1306   42]
 [  46   15]]
NEK9_inhibition_MFP_none_scaled NEK9_binding
accuracy: 0.9588, precision: 0.6364, recall: 0.1148, specificity: 0.9970, cm: [[1344    4]
 [  54    7]]
NEK9_inhibition_MFP_none_scaled NEK9_inhibition
accuracy: 0.8931, precision: 0.0000, recall: 0.0000, specificity: 1.0000, cm: [[351   0]
 [ 42   0]]
NEK9_inhibition_MFP_none_scaled NEK9_inhibition
accuracy: 0.9618, precision: 0.7547, recall: 0.9524, specificity: 0.9630, cm: [[338  13]
 [  2  40]]
NEK9_inhibition_MFP_none_scaled NEK9_inhibition
accuracy: 0.9898, precision: 0.9524, recall: 0.9524, specificity: 0.9943, cm: [[349   2]
 [  2  40]]


/Users/jayceepang/miniconda3/envs/ATOM_CODE/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/jayceepang/miniconda3/envs/ATOM_CODE/lib/python3.9/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/Users/jayceepang/miniconda3/envs/ATOM_CODE/lib/python3.9/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/Users/jayceepang/miniconda3/envs/ATOM_CODE/lib/python3.9/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-05 to the diagonal
  warnings.warn(
/Users/jayceepang/miniconda3/envs/ATOM_CODE/lib/python3.9/site-packages/linear_operato